In [31]:
import networkx as nx
import pandas as pd
import numpy as np
import sys
import torch
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, ChebConv
import matplotlib.pyplot as plt

In [32]:
dataset_path = 'simulator_data_1.csv'

In [33]:
# Load the dataset
# dataset_path = "path_to_your_dataset.csv"  # Replace with your actual dataset path
data = pd.read_csv(dataset_path)

In [34]:
import pandas as pd
import torch
from torch_geometric.data import Data



# Function to calculate distance (mock-up for illustration)
def calculate_distance(x1, y1, x2, y2):
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5

# Modify your function to process each timestamp as follows:
def create_graph_data_for_timestamp(df, timestamp,  vehicle_ids, tower_ids):
    timestamp_data = df[df['timestamp'] == timestamp]


    # Map specific vehicle and tower IDs to consecutive indices
    vehicle_mapping = {vid: i for i, vid in enumerate(vehicle_ids)}
    tower_mapping = {tid: i + len(vehicle_ids) for i, tid in enumerate(tower_ids)}

    node_features = []
    edge_index = []
    edge_features = []

    # Process vehicle nodes
    for vid in vehicle_ids:
        vehicle_data = timestamp_data[timestamp_data['vehicleId'] == vid]
        if not vehicle_data.empty:
            avg_speed = vehicle_data['vehicleSpeed'].mean()
            avg_dir = vehicle_data['vehicleDirection'].mean()
            avg_pos_x = vehicle_data['vehiclePosX'].mean()
            avg_pos_y = vehicle_data['vehiclePosY'].mean()
            vehicle_features = [avg_speed, avg_dir, avg_pos_x, avg_pos_y]
        else:
            # Provide some default or mean features if the vehicle is not present
            vehicle_features = [0, 0, 0, 0]  # Placeholder, adjust as needed
        node_features.append(vehicle_features)

    # Process tower nodes
    for tid in tower_ids:
        tower_data = timestamp_data[timestamp_data['masterId'] == tid]
        if not tower_data.empty:
            avg_pos_x = tower_data['masterPosX'].mean()
            avg_pos_y = tower_data['masterPosY'].mean()
            avg_load = tower_data['masterLoad'].mean()
        else:
            # Default or mean features for absent towers
            avg_pos_x = avg_pos_y = avg_load = 0  # Placeholder
        tower_features = [avg_pos_x, avg_pos_y, avg_load, 0.0]  # Last value is placeholder
        node_features.append(tower_features)

    # Add edges for existing pairs in this timestamp
    for _, row in timestamp_data.iterrows():
        if row['vehicleId'] in vehicle_mapping and row['masterId'] in tower_mapping:
            vehicle_id = vehicle_mapping[row['vehicleId']]
            tower_id = tower_mapping[row['masterId']]
            edge_index.append([vehicle_id, tower_id])
            edge_features.append([row['throughput'], row['masterRssi'], row['distance']])

    node_features_tensor = torch.tensor(node_features, dtype=torch.float)
    edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_features_tensor = torch.tensor(edge_features, dtype=torch.float)

    return Data(x=node_features_tensor, edge_index=edge_index_tensor, edge_attr=edge_features_tensor)

# Load your dataset
# df = data
# timestamps = sorted(df['timestamp'].unique())
# graph_datasets = [create_graph_data_for_timestamp(df, ts) for ts in timestamps]


In [35]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class TemporalGNN(nn.Module):
    def __init__(self, node_features_dim, edge_features_dim, memory_dim):
        super(TemporalGNN, self).__init__()
        self.conv = GCNConv(node_features_dim, memory_dim)
        self.edge_predictor = nn.Sequential(
            nn.Linear(2 * memory_dim, 2 * memory_dim),
            nn.ReLU(),
            nn.Linear(2 * memory_dim, edge_features_dim)
        )

    def forward(self, x, edge_index):
        # Generate node embeddings using GCN
        node_embeddings = self.conv(x, edge_index)
        # Combine embeddings of connected nodes to form edge features
        src, dest = edge_index
        edge_features = torch.cat([node_embeddings[src], node_embeddings[dest]], dim=1)
        # Predict edge features from combined node embeddings
        predicted_edge_features = self.edge_predictor(edge_features)
        return predicted_edge_features

def predictive_loss(predicted_edge_features, predicted_edge_index, actual_edge_features, actual_edge_index):
    # Convert edge indices to set of tuples for comparison
    predicted_edges = set(map(tuple, predicted_edge_index.t().cpu().tolist()))
    actual_edges = set(map(tuple, actual_edge_index.t().cpu().tolist()))

    # Find common edges between predicted and actual
    common_edges = predicted_edges.intersection(actual_edges)

    # Extract features for common edges from predictions and actuals
    pred_features = [predicted_edge_features[i] for i, edge in enumerate(predicted_edge_index.t().tolist()) if tuple(edge) in common_edges]
    actual_features = [actual_edge_features[i] for i, edge in enumerate(actual_edge_index.t().tolist()) if tuple(edge) in common_edges]

    # If no common edges, return zero loss with gradients
    if not pred_features or not actual_features:
        return torch.tensor(0.0, requires_grad=True).to(predicted_edge_features.device)

    # Stack features and calculate MSE loss
    pred_features = torch.stack(pred_features)
    actual_features = torch.stack(actual_features)
    return torch.nn.functional.mse_loss(pred_features, actual_features)


In [36]:
# import torch
df = data
# Convert timestamps to numeric and sort
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
df.sort_values('timestamp', inplace=True)

# Define training and testing ranges
train_end_time = 8  # Define the timestamp up to which data should be used for training
test_start_time = 8.5  # Define the timestamp from which data should be used for testing

# Select train and test data based on timestamps
train_data = df[df['timestamp'] <= train_end_time]
test_data = df[df['timestamp'] == test_start_time]

# Example of creating graph data for each set
vehicle_ids = sorted(df['vehicleId'].unique())  # Ensure you have a list of all vehicle IDs
tower_ids = sorted(df['towerId'].unique())  # Ensure you have a list of all tower IDs

# Function to create graph data (assuming it's already defined)
train_graphs = [create_graph_data_for_timestamp(train_data, ts, vehicle_ids, tower_ids) for ts in train_data['timestamp'].unique()]
test_graphs = [create_graph_data_for_timestamp(test_data, ts, vehicle_ids, tower_ids) for ts in test_data['timestamp'].unique()]


model = TemporalGNN(node_features_dim=4, edge_features_dim=3, memory_dim=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epoch = 100
# Training loop
for epoch in range(num_epoch):  # Number of training epochs
    print(f"Starting Epoch {epoch+1}")
    for graph_data in train_graphs:
        optimizer.zero_grad()
        predicted_edge_features = model(graph_data.x, graph_data.edge_index)
        loss = predictive_loss(predicted_edge_features, graph_data.edge_index, graph_data.edge_attr, graph_data.edge_index)
        loss.backward()
        optimizer.step()
    print(f"Completed training for Epoch {epoch+1} ")

# # Testing loop
# model.eval()
# with torch.no_grad():
#     for graph_data in test_graphs:
#         predicted_edge_features = model(graph_data.x, graph_data.edge_index)
#         print(f"Predicted edge features for timestamp {test_start_time}: {predicted_edge_features}")


# model = TemporalGNN(node_features_dim=4, edge_features_dim=3, memory_dim=64)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# # Define the training range for timestamps 2 to 6 (indices 1 to 5 in zero-indexed Python lists)
# train_start_index = 5
# train_end_index = 7

# # Define the index for prediction which is timestamp 7 (index 6)
# prediction_index = 6

# # Training the model
# for epoch in range(50):  # Number of epochs
#     total_loss = 0
#     count = 0  # Count of batches that actually contribute to the loss
#     for i in range(train_start_index, train_end_index + 1):  # Loop from timestamp 2 to 6
#         optimizer.zero_grad()
#         current_data = graph_datasets[i]
#         next_data = graph_datasets[i + 1]  # Get the next timestamp data
#         predicted_edge_features = model(current_data.x, current_data.edge_index)
#         loss = predictive_loss(predicted_edge_features, current_data.edge_index, next_data.edge_attr, next_data.edge_index)
#         if loss.requires_grad:  # Only backpropagate if the loss is not a placeholder
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#             count += 1
#     if count > 0:
#         print(f"Epoch {epoch}, Avg Loss: {total_loss / count}")
#     else:
#         print(f"Epoch {epoch}, No valid training data")

# # Prediction
# model.eval()
# with torch.no_grad():
#     test_data = graph_datasets[prediction_index]  # Using the data of the timestamp 7 for prediction
#     predicted_edge_features = model(test_data.x, test_data.edge_index)
#     print(f"Predicted edge features for timestamp {prediction_index+1}:", predicted_edge_features)


Starting Epoch 1
Completed training for Epoch 1 
Starting Epoch 2
Completed training for Epoch 2 
Starting Epoch 3
Completed training for Epoch 3 
Starting Epoch 4
Completed training for Epoch 4 
Starting Epoch 5
Completed training for Epoch 5 
Starting Epoch 6
Completed training for Epoch 6 
Starting Epoch 7
Completed training for Epoch 7 
Starting Epoch 8
Completed training for Epoch 8 
Starting Epoch 9
Completed training for Epoch 9 
Starting Epoch 10
Completed training for Epoch 10 
Starting Epoch 11
Completed training for Epoch 11 
Starting Epoch 12
Completed training for Epoch 12 
Starting Epoch 13
Completed training for Epoch 13 
Starting Epoch 14
Completed training for Epoch 14 
Starting Epoch 15
Completed training for Epoch 15 
Starting Epoch 16
Completed training for Epoch 16 
Starting Epoch 17
Completed training for Epoch 17 
Starting Epoch 18
Completed training for Epoch 18 
Starting Epoch 19
Completed training for Epoch 19 
Starting Epoch 20
Completed training for Epoch 20

In [27]:


def create_complete_bipartite_edge_index(vehicle_ids, tower_ids, vehicle_mapping, tower_mapping):
    # Generate all possible combinations of vehicles and towers
    edge_index = []
    for vehicle_id in vehicle_ids:
        for tower_id in tower_ids:
            src = vehicle_mapping[vehicle_id]
            dest = tower_mapping[tower_id]
            edge_index.append([src, dest])
    # Convert list to tensor
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return edge_index

    # Example model invocation for the full set of potential edges
def predict_all_edges(model, node_features, complete_edge_index):
    model.eval()
    with torch.no_grad():
        predicted_edge_features = model(node_features, complete_edge_index)
    return predicted_edge_features



def map_predictions_to_ids(predicted_edge_features, edge_index, vehicle_mapping, tower_mapping):
    mapped_predictions = {}
    # Reverse the mappings to map index back to ID
    index_to_vehicle = {v: k for k, v in vehicle_mapping.items()}
    index_to_tower = {v: k for k, v in tower_mapping.items()}

    # Loop through each edge and map the prediction to the corresponding vehicle and tower IDs
    for i, (src, dest) in enumerate(edge_index.t().tolist()):
        vehicle_id = index_to_vehicle.get(src, None)
        tower_id = index_to_tower.get(dest - len(vehicle_mapping), None)  # Adjust index by the number of vehicles if necessary

        if vehicle_id is not None and tower_id is not None:
            mapped_predictions[(vehicle_id, tower_id)] = predicted_edge_features[i].tolist()

    return mapped_predictions

# Example execution within a testing loop for graphs
# for graph_data in test_graphs:
#     model.eval()
#     with torch.no_grad():
#         predicted_edge_features = model(graph_data.x, graph_data.edge_index)


In [29]:
vehicle_mapping = {vid: i for i, vid in enumerate(vehicle_ids)}

# Create mapping from tower ID to a unique index
tower_mapping = {tid: i for i, tid in enumerate(tower_ids)}

vehicle_ids = list(vehicle_mapping.keys())
tower_ids = list(tower_mapping.keys())

# Create a complete bipartite edge index
complete_edge_index = create_complete_bipartite_edge_index(vehicle_ids, tower_ids, vehicle_mapping, tower_mapping)

# Predict edge features for all possible edges
predicted_edge_features = predict_all_edges(model, graph_data.x, complete_edge_index)

# Map predictions back to vehicle and tower IDs
mapped_predictions = map_predictions_to_ids(predicted_edge_features, complete_edge_index, vehicle_mapping, tower_mapping)
print(mapped_predictions)
# Print the mapped predictions
for pair, features in mapped_predictions.items():
    print(f"Vehicle {pair[0]} and Tower {pair[1]} -> Predicted Features: {features}")

{}


In [ ]:
#for each vehicle select the best towers
#return the estimated metrics for extended ho decision-making
#check the number of edges

#tune the metrics and history size